In [1]:
#Setting up environment for Apache Spark
import os
import sys

os.chdir(r"C:\spark\spark-files")
os.curdir

if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = 'C:\spark'

SPARK_HOME = os.environ['SPARK_HOME']

sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.4-src.zip"))

In [2]:
#importing dependencies
from pyspark import SparkConf, SparkContext
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF
import operator
import sys

In [3]:
#Taking search string as input from user when integrated to web
list = sys.argv
st = str(list[1])
wo = st.lower().split()

#Temporarily taking manual input here, for testing
st= "Abraham Lincoln Gettysburg"
wo = st.split()
print(wo)

['Abraham', 'Lincoln', 'Gettysburg']


In [4]:
#Setting up Spark Context, Run only once in a single session

conf = SparkConf().setMaster("local").setAppName("SparkTFIDF")
sc = SparkContext(conf = conf)

In [5]:
#loading and extracting data

rawData = sc.textFile("wikipedia_data.tsv") #Located at C:\spark\spark-files
fields = rawData.map(lambda x: x.split("\t"))
documents = fields.map(lambda x: x[3].split(" "))
documentNames = fields.map(lambda x: x[1])

#rawData.take(1)

In [6]:
#Calculating TF-IDF

hashingTF = HashingTF(100000)  
article_hash_value = hashingTF.transform(documents)
article_hash_value.cache() #storing value in-memory

PythonRDD[2] at RDD at PythonRDD.scala:48

In [7]:
idf = IDF(minDocFreq=2).fit(article_hash_value)
tfidf = idf.transform(article_hash_value)
tfidf.cache()

MapPartitionsRDD[9] at mapPartitions at PythonMLLibAPI.scala:1335

In [9]:
def df_of_given_word(article_hash_value, word_hash):
    gtbTF = hashingTF.transform([word_hash])
    gtbHashValue1 = int(gtbTF.indices[0])
    gtbRelevance1 = tfidf.map(lambda x: x[gtbHashValue1] )
    y= [] 
    for z in gtbRelevance1.toLocalIterator():
        y.append(z)    
    return y

In [15]:
b = []
for i in range(len(wo)):
    c = []
    c= df_of_given_word(article_hash_value,wo[i])
    if i == 0:
        b = c
    else:
        b = map(operator.add, b,c)
#print(b)

In [14]:
dn=[]
for z in documentNames.toLocalIterator():
#for z in documentNames.collect():
        dn.append(z)
zipp = map(lambda x,y: (x,y),dn,b) 
# print(list(zipp))
zipp.sort(key=lambda tup: tup[1], reverse = True)
zipp.sort(key=lambda tup: tup[1], reverse = True)
print("Best document for " +st+  " is:")
print(zipp[0])

Best document for Abraham Lincoln Gettysburg is:
(u'Abraham Lincoln', 683.64280658581845)
